In [53]:
import uuid
from typing import Dict
from datetime import datetime
from pydantic import BaseModel
from langchain.memory import ConversationBufferMemory
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

# groq llm client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [9]:
def groq_chat(messages: list) -> str:

    # if SessionId.session_id not in session_state:
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{messages}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    return response.choices[0].message.content.strip()

In [10]:
groq_chat(messages=["what is the capital of India?"])

'The capital of India is New Delhi.'

In [11]:
class ChatRequest(BaseModel):
    query: str
    session_id: str = None

class ChatResponse(BaseModel):
    response: str
    session_id: str = None

In [12]:
# --- Memory Management ---
# In-memory active sessions
active_sessions: Dict[str, ConversationBufferMemory] = {}

In [13]:
active_sessions

{}

In [14]:
# --- Session filter/create function ---
def filter_or_create_session_id(session_id: str) -> str:
    """
    Ensures session_id exists in active_sessions.
    If not, creates a new one with ConversationBufferMemory.
    """

    if session_id not in active_sessions:
        session_id = str(uuid.uuid4().hex)
        active_sessions[session_id] = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        print(f"🆕 Created new session: {session_id}")
    else:
        print(f"✅ Using existing session: {session_id}")

    return session_id

In [15]:
filter_or_create_session_id(session_id="ajhflksjloejojposgsg")

🆕 Created new session: 7cdaca33589344f38259f96b84615d87


C:\Users\Admin\AppData\Local\Temp\ipykernel_7064\176814929.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  active_sessions[session_id] = ConversationBufferMemory(


'7cdaca33589344f38259f96b84615d87'

In [16]:
active_sessions

{'7cdaca33589344f38259f96b84615d87': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [17]:
# ===============m Filter or create session id ========================
def filter_or_create_session_id(session_id: str) -> str:

    if session_id not in active_sessions:
        # Create new session
        session_id = str(uuid.uuid4().hex)
        active_sessions[session_id] = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    else:
        # Use existing session
        return session_id
    
# filter or create sesison id
filtered_session_id = filter_or_create_session_id(session_id="fda9441bb78e46a79398ff0095cc8593")
print(filtered_session_id)

None


In [18]:
active_sessions

{'7cdaca33589344f38259f96b84615d87': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 '195c3d45f54a47bebdcfa002e8ed6ec4': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [19]:
# session_id = "fda9441bb78e46a79398ff0095cc8593"

# if filter_or_create_session_id(SessionId(session_id=session_id)):
#     print(session_id)
# else:
#     print("created session id")

In [20]:
def chatbot(query: str) -> str:
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{query}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    return response.choices[0].message.content.strip()

In [21]:
response = chatbot(query="what is the capital of India?")
print(response)

The capital of India is New Delhi.


<!-- ### Now manage the session with memeory in chatbot -->

In [22]:
active_sessions

{'7cdaca33589344f38259f96b84615d87': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 '195c3d45f54a47bebdcfa002e8ed6ec4': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [23]:
session_id = "fda9441bb78e46a79398ff0095cc8593"

if filter_or_create_session_id(session_id=session_id):
    print(session_id)
else:
    print("created session id")

created session id


In [24]:
active_sessions

{'7cdaca33589344f38259f96b84615d87': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 '195c3d45f54a47bebdcfa002e8ed6ec4': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 'f9329964da4141eba03f1db9214b4242': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [32]:
class ChatRequest(BaseModel):
    query: str
    session_id: str = None  # Optional, if not provided, create new

class ChatResponse(BaseModel):
    session_id: str = None  # Return session_id in response
    history: list = []  # Chat history

In [27]:
active_sessions['7cdaca33589344f38259f96b84615d87'].chat_memory.add_user_message
active_sessions['7cdaca33589344f38259f96b84615d87'].chat_memory.add_ai_message

<bound method BaseChatMessageHistory.add_ai_message of InMemoryChatMessageHistory(messages=[])>

In [30]:
# --- Chatbot function with conversation memory ---
def chat_with_memory(state: ChatRequest) -> ChatResponse:
    """
    Manage conversational chat sessions with memory.
    Creates or reuses a session and keeps conversation context.
    """
    # 1. Ensure session exists
    session_id = filter_or_create_session_id(session_id=state.session_id)
    memory = active_sessions[session_id]

    # 2. Add user message to memory
    memory.chat_memory.add_user_message(state.query)

    # 3. Convert chat memory to Groq messages format
    messages_for_groq = [
        {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
        for msg in memory.chat_memory.messages]

    # 4. Get response from Groq model
    assistant_reply = chatbot(query=messages_for_groq)

    # 5. Add assistant reply to memory
    memory.chat_memory.add_ai_message(assistant_reply)

    # 6. Prepare history for response
    chat_history = [
        {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
        for msg in memory.chat_memory.messages
    ]
    
    # 7. Return response
    return ChatResponse(session_id=session_id, history=chat_history)

In [42]:
initial_state = ChatRequest(**{"query": "hello my name is jeet", "session_id": "7cdaca33589344f38259f96b84615d87"})
initial_state

ChatRequest(query='hello my name is jeet', session_id='7cdaca33589344f38259f96b84615d87')

In [43]:
response = chat_with_memory(initial_state)

In [48]:
response.history

[{'role': 'user', 'content': 'hello my name is jeet'},
 {'role': 'assistant', 'content': 'Hello Jeet. How can I assist you today?'}]

In [50]:
initial_state = ChatRequest(**{"query": "what is my name?", "session_id": "7cdaca33589344f38259f96b84615d87"})
response = chat_with_memory(initial_state)
print(response.history)

[{'role': 'user', 'content': 'hello my name is jeet'}, {'role': 'assistant', 'content': 'Hello Jeet. How can I assist you today?'}, {'role': 'user', 'content': 'what is my name?'}, {'role': 'assistant', 'content': 'Hello Jeet. Your name is Jeet.'}]


In [51]:
initial_state = ChatRequest(**{"query": "what did i ask just before?", "session_id": "7cdaca33589344f38259f96b84615d87"})
response = chat_with_memory(initial_state)
print(response.history)

[{'role': 'user', 'content': 'hello my name is jeet'}, {'role': 'assistant', 'content': 'Hello Jeet. How can I assist you today?'}, {'role': 'user', 'content': 'what is my name?'}, {'role': 'assistant', 'content': 'Hello Jeet. Your name is Jeet.'}, {'role': 'user', 'content': 'what did i ask just before?'}, {'role': 'assistant', 'content': 'You asked "what is my name?" just before.'}]


In [52]:
for msg in response.history:
    print(msg)

{'role': 'user', 'content': 'hello my name is jeet'}
{'role': 'assistant', 'content': 'Hello Jeet. How can I assist you today?'}
{'role': 'user', 'content': 'what is my name?'}
{'role': 'assistant', 'content': 'Hello Jeet. Your name is Jeet.'}
{'role': 'user', 'content': 'what did i ask just before?'}
{'role': 'assistant', 'content': 'You asked "what is my name?" just before.'}
